In [ ]:
pip install underthesea

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
import string
import unicodedata
import re
import underthesea
from collections import Counter

In [9]:
# Đọc dữ liệu từ file CSV
data = pd.read_csv("facebook_comment_2k7.csv")

# Tiền xử lý dữ liệu văn bản tiếng Việt
def preprocess_text_vietnamese(text):
    # Chuyển văn bản về chữ thường
    text = text.lower()
    # Loại bỏ dấu câu
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Loại bỏ dấu ký tự đặc biệt
    text = re.sub(r'[^\w\s]', '', text)
    # Tách từ tiếng Việt
    words = underthesea.word_tokenize(text)
    # Ghép từ lại thành câu
    text = ' '.join(words)
    return text

In [10]:
# Define models
models = {
    "Logistic Regression": make_pipeline(TfidfVectorizer(preprocessor=preprocess_text_vietnamese, max_features=10000), LogisticRegression()),
    "Naive Bayes": make_pipeline(TfidfVectorizer(preprocessor=preprocess_text_vietnamese, max_features=10000), MultinomialNB()),
    "Random Forest": make_pipeline(TfidfVectorizer(preprocessor=preprocess_text_vietnamese, max_features=10000), RandomForestClassifier()),
    "SVM": make_pipeline(TfidfVectorizer(preprocessor=preprocess_text_vietnamese, max_features=10000), SVC(kernel='linear')),
    "KNeighbors": make_pipeline(TfidfVectorizer(preprocessor=preprocess_text_vietnamese, max_features=10000), KNeighborsClassifier(n_neighbors=5)),
    "MLP": make_pipeline(TfidfVectorizer(preprocessor=preprocess_text_vietnamese, max_features=10000), MLPClassifier()),
    "Gradient Boosting": make_pipeline(TfidfVectorizer(preprocessor=preprocess_text_vietnamese, max_features=10000), GradientBoostingClassifier())
}

In [11]:
# Train models
for name, model in models.items():
    model.fit(data['text'], data['label'])

In [12]:
# Define predict_labels function
def predict_labels(input_sentence):
    # Predict labels using all models
    predictions = {name: model.predict([input_sentence])[0] for name, model in models.items()}

    # Count label occurrences
    label_counter = Counter(predictions.values())

    # Get most common labels
    most_common_labels = [label for label, count in label_counter.items() if count >= 4]

    # Return "Uncertain" if no label is predicted more than 4 times
    if not most_common_labels:
        return ["Uncertain"]

    return most_common_labels

In [13]:
# Get user input
input_sentence = input("Nhập câu: ")

# Predict labels for the input sentence
predicted_labels = predict_labels(preprocess_text_vietnamese(input_sentence))
print("Dự đoán nhãn:", predicted_labels)


Nhập câu: hi em
Dự đoán nhãn: ["['normal']"]
